# Chapter 1: The Python Data Model

In this chapter, we'll see how using the special methods in Python, e.g. `__len__` and `__getitem__`, allows us to implement familiar Python syntax in our classes for powerful, readable code.

## A Pythonic card deck

We can leverage the standard library of Python packages to build classes quickly and effectively. Below, we construct a basic playing card class using `collections.namedtuple`. Then we use this simple class as a building block to construct a full deck of cards.

### Example 1.1: A deck of playing cards

In [1]:
# Attributes-only class using namedtuple
import collections
Card = collections.namedtuple('Card', ['rank', 'suit'])

In [2]:
ace_of_spades = Card('A', 'spades')
print(ace_of_spades)

Card(rank='A', suit='spades')


Now let's use this simple class to build a deck of cards, `Deck`, which we can cycle through like a Python list. 

To allow for functionality such as `Deck[0] = Card(rank='A', suit='spades')`, we need to define the `__len__` and `__getitem__` special methods. 

`__len__` is necessary so that the class understands its length, for the purpose of e.g. `Deck[-1]`. 

Defining `__getitem__` allows us to evaluate the deck at any position, with e.g. slicing functionality.

In [3]:
# Deck class
class Deck:
    ranks: list[str] = [str(j) for j in range(2, 11)] + list('JQKA')
    suits: list[str] = 'spades hearts diamonds clubs'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [4]:
deck = Deck()
print(deck[0]) # first card
print(deck[:4]) # first four cards
print(deck[-1]) # last card

Card(rank='2', suit='spades')
[Card(rank='2', suit='spades'), Card(rank='3', suit='spades'), Card(rank='4', suit='spades'), Card(rank='5', suit='spades')]
Card(rank='A', suit='clubs')


**Note:** as it stands, the `Deck` class cannot be shuffled as it is **immutable**: the position of cards cannot be changed - they are locked away in `Deck._cards`, and handling this attribute directly is a violation of encapsulation. Chapter 13 of Fluent Python solves this problem with the `__setitem__` special method, but this is beyond the scope of the Bootcamp.

Let's write a function `card_strength` to determine the value of a card. For example, we should have `strength(Card('2', 'clubs')) = 1` and `strength(Card('2', 'diamonds')) = 2`, and so forth, all the way up to `strength(Card('A', 'spades')) = 52`.

In [5]:
# Value add-on for suit
suit_strength: dict = dict(spades = 4, hearts = 3, diamonds = 2, clubs = 1)

# Strength of a card
def strength(card: Card) -> int:
    return 4 * Deck.ranks.index(card.rank) + suit_strength[card.suit]

Now we can sort the deck in strength order with `sorted`:

In [6]:
# Sorted deck
for card in sorted(deck, key=strength):
    print(f'{card} has {strength(card)} strength')

Card(rank='2', suit='clubs') has 1 strength
Card(rank='2', suit='diamonds') has 2 strength
Card(rank='2', suit='hearts') has 3 strength
Card(rank='2', suit='spades') has 4 strength
Card(rank='3', suit='clubs') has 5 strength
Card(rank='3', suit='diamonds') has 6 strength
Card(rank='3', suit='hearts') has 7 strength
Card(rank='3', suit='spades') has 8 strength
Card(rank='4', suit='clubs') has 9 strength
Card(rank='4', suit='diamonds') has 10 strength
Card(rank='4', suit='hearts') has 11 strength
Card(rank='4', suit='spades') has 12 strength
Card(rank='5', suit='clubs') has 13 strength
Card(rank='5', suit='diamonds') has 14 strength
Card(rank='5', suit='hearts') has 15 strength
Card(rank='5', suit='spades') has 16 strength
Card(rank='6', suit='clubs') has 17 strength
Card(rank='6', suit='diamonds') has 18 strength
Card(rank='6', suit='hearts') has 19 strength
Card(rank='6', suit='spades') has 20 strength
Card(rank='7', suit='clubs') has 21 strength
Card(rank='7', suit='diamonds') has 22 

In general, any iterable can be sorted in an order determined by a function, `key`, with the `sorted(iterable, key)` method. By default this order is ascending, but you can add the `reverse=True` attribute for descending order.

You start to feel that, even though this example involves a basic deck of cards, such Pythonic syntax can be used for much more powerful examples in programming.

## How special methods are used

Special methods are meant to be called by the Python interpreter. Whilst you will implement special methods frequently, you seldom call them directly. For example, you don't write `Deck.__len__()` but instead `len(Deck)`.

### Emulating numeric types

Let's demonstrate the power of special methods further with another simple example: creating a 2D `Vector` class that behaves exactly as we would expect.

First, let's design the API for the class by writing a simple test that we hope to hold true for the solution:

```
>>> v = Vector(3, 4)
>>> w = Vector(4, -1)

>>> v + w
>>> Vector(7, 3)

>>> abs(v)
>>> 5.0

>>> v * 3
>>> Vector(9, 12)
```

In particular, note that the `+` operator works with `Vector` classes, and returns a `Vector`.

### Example 1.2: A two-dimensional vector class

In [8]:
from math import sqrt

# Vector class
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    # String representation in the console
    def __repr__(self):
        return f'Vector({self.x}, {self.y})'
    
    # Adding
    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    # Scalar multiplication
    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)
        '''
        NOTE: this implementation only allows v * 3. Writing 3 * v will NOT work.
        This can be fixed with the special method __rmul__, explored in Fluent Python Chapter 13.
        '''

    # Modulus
    def __abs__(self):
        return sqrt(self.x**2 + self.y**2)
    
    # Return false for zero vector
    def __bool__(self):
        return bool(abs(self))
        '''
        NOTE: A faster way of writing this is 
        return bool(self.x or self.y)
        as you avoid using abs, sqrt and squaring. But this way is less readable.
        We also need to use bool(x or y) here: writing (x or y) returns an object e.g. x if bool(x) is True. 
        '''

There are a few new special methods here. The mathematical ones should feel intuitive, but let's discuss `__repr__` and `__bool__`.
 
`__repr__` is used to provide a string representation of the class object when called in the console. An object would otherwise show as e.g. `<Vector object at 0x10e100080>`. The convention with string representations is to make them unambiguous and match the code required to create the object in the first place. For example, calling `v` from earlier would give `Vector(3, 4)`, which is exactly what we would write in the console to construct such an object.

`__bool__` is used to determine whether instances of a user-defined class are "true-like" or "false-like". For our `Vector` class we have defined `__bool__` in such a way that intuitively follows from the idea of 0 = False and 1 = True. In general, `bool(object)` is calculated by first checking if `object.__bool__()` is defined: if not, then Python tries `object.__len__()`. If this returns 0, then `bool(object)` returns `False`. If not, or if `__len__` is also undefined for the object, Python returns `True`.

**Note**: Another special method, `__str__`, is implicitly used by the print function and should return a string suitable for displaying to end users. This special method could be used to give further (small) amounts of detail, but is not crucial. `__str__` will use `__repr__` as a fallback if `__str__` is not defined. 

In [12]:
v = Vector(3, 4)
w = Vector(4, -1)

print(v + w)
print(abs(v))
print(v * 3)

Vector(7, 3)
5.0
Vector(9, 12)


**Challenge**: extend `Vector` to $n$ dimensions.

In [15]:
class VectorND:
    # TODO
    pass

## Summary

- Implementing special methods can make your objects behave like Python built-in types
- A basic requirement for a Python object is to provide usable string representations of itself for debugging and logging (using `__repr__`) and presentation to end-users (`__str__`)
- Emulating sequences, as done earlier in the `Deck` class, is one of the most common applications of special methods

## Further reading

- [The Python Language Reference - Data Model chapter](https://docs.python.org/3/reference/datamodel.html)